In [280]:
import sys
sys.path.append('..')

In [281]:
from preprocessing import  clean_rawdata, resolve_synonyms, scaling
from model import train, predict
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from itertools import permutations, combinations

In [315]:
df = pd.read_csv("../raw_data/wine_data_csv.csv", index_col=0)
df.drop_duplicates(inplace=True)
columns_to_drop=['designation','region_2','taster_name','taster_twitter_handle','region_1']
df.drop(columns=columns_to_drop,inplace=True)
df.dropna(inplace=True)
df.drop(df[df['type']=='delete'].index,inplace=True)

In [316]:
syns_raw = pd.read_csv("../raw_data/wine_synonyms_csv.csv", index_col=0)
# Extracting the single column from the synonyms files that has all synonyns for each row
all_grape_names = syns_raw.NAME_ALL.str.split(', ')
# The synonyms file has multiple rows (synonyms) that hasn't been unified (synonymised) properly.
# Here a table of synonyms is created to consolidate all synonyms.
syns = {}     # for each item: key will be main synonym, values will include all synonyms including the main one (used for key)
for row in all_grape_names:
    flat_dict = [num for elem in list(syns.values()) for num in elem]
    # checking if synonyms in each row are already present is the dictionary being created
    # if not, it creates a key and values
    if any(grape in flat_dict for grape in row) == False:
        syns[row[0]] = row
    # if yes, adds the new synonyns that don't exist yet in the list
    else:
        res = next((sub for sub in syns if any(grape in syns[sub] for grape in row) == True), None)
        syns[res].extend([item for item in row if item not in syns[res]])
del flat_dict
# Using the unified synonym table, it populates the column for grape vaiety in the main data table
# with the main synonyms for each grape variety
df['variety_adj'] = df['variety'].apply(lambda grape: next((key for key, value in syns.items() if grape in value), None))
df.drop(columns=['variety'], inplace=True)
df

,country,description,points,price,province,title,winery,type,region,variety_adj
ID,,,,,,,,,,
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Quinta dos Avidagos,red,Douro,Portuguese Red
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Rainstorm,white,Willamette Valley,Pinot Gris
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,St. Julian,white,Lake Michigan Shore,Riesling
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Sweet Cheeks,red,Willamette Valley,Pinot Noir
5,Spain,Blackberry and raspberry aromas show a typical...,87,15.0,Northern Spain,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tandem,red,Navarra,Tempranillo-Merlot
...,...,...,...,...,...,...,...,...,...,...
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,90,28.0,Mosel,Dr. H. Thanisch (Erben MÃ¼ller-Burggraef) 2013...,Dr. H. Thanisch (Erben MÃ¼ller-Burggraef),white,Mosel,Riesling
129967,US,Citation is given as much as a decade of bottl...,90,75.0,Oregon,Citation 2004 Pinot Noir (Oregon),Citation,red,Oregon,Pinot Noir
129968,France,Well-drained gravel soil gives this wine its c...,90,30.0,Alsace,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Domaine Gresser,white,Alsace,Gewurztraminer


In [317]:
df['type'].value_counts()

type
red          68055
white        35851
sparkling     3857
rose          3773
Name: count, dtype: int64

In [318]:
X_scaled = scaling(df)
X_scaled

Index(['country', 'description', 'points', 'price', 'province', 'title',
       'winery', 'type', 'region', 'variety_adj'],
      dtype='object')


,country,points,price,province,winery,region,variety_adj,type_red,type_rose,type_sparkling,type_white
0,-0.24,-0.2,-0.40,-0.262931,0.968039,-1.133721,0.029091,3.0,0.0,0.0,0.0
1,0.12,-0.2,-0.44,0.422414,0.983822,2.591860,-0.014545,0.0,0.0,0.0,3.0
2,0.12,-0.2,-0.48,0.206897,1.330001,-0.401163,0.116364,0.0,0.0,0.0,3.0
3,0.12,-0.2,1.60,0.422414,1.388925,2.591860,0.000000,3.0,0.0,0.0,0.0
4,0.00,-0.2,-0.40,0.396552,1.409970,0.502326,0.396364,3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
111531,-0.80,0.4,0.12,0.267241,-0.626858,0.247674,0.116364,0.0,0.0,0.0,3.0
111532,0.12,0.4,2.00,0.422414,-1.077732,0.655814,0.000000,3.0,0.0,0.0,0.0
111533,-0.88,0.4,0.20,-0.672414,-0.793108,-2.717442,-0.636364,0.0,0.0,0.0,3.0
111534,-0.88,0.4,0.28,-0.672414,-0.770485,-2.717442,-0.014545,0.0,0.0,0.0,3.0


In [319]:
neigh = NearestNeighbors(n_neighbors=len(df), algorithm='ball_tree', n_jobs=-1)
neigh.fit(X_scaled)

NearestNeighbors(algorithm='ball_tree', n_jobs=-1, n_neighbors=111536)

In [320]:
# input_list=[1, 28, 390, 46767, 55757] # that was a random list only for testing
input_list=[482, 33, 1338, 106274, 8090] # Jaime's list

In [321]:
input_df=X_scaled.iloc[input_list]
input_df

,country,points,price,province,winery,region,variety_adj,type_red,type_rose,type_sparkling,type_white
482,0.12,0.6,0.44,-0.500000,0.910430,-2.148837,-0.603636,3.0,0.0,0.0,0.0
33,-0.60,-0.4,-0.56,0.737069,-0.455347,0.983721,0.061818,3.0,0.0,0.0,0.0
1338,0.12,1.0,0.52,-0.500000,-0.616862,-1.126744,0.061818,3.0,0.0,0.0,0.0
106274,-1.44,0.0,-0.36,0.198276,-1.085624,0.000000,-0.290909,3.0,0.0,0.0,0.0
8090,0.12,-0.2,2.40,-0.500000,-1.543864,-1.956977,-0.272727,3.0,0.0,0.0,0.0


In [322]:
input_df_mean=np.mean(input_df, axis=0)
input_df_mean

country          -0.336000
points            0.200000
price             0.488000
province         -0.112931
winery           -0.558253
region           -0.849767
variety_adj      -0.208727
type_red          3.000000
type_rose         0.000000
type_sparkling    0.000000
type_white        0.000000
dtype: float64

In [323]:
distances, indices = neigh.kneighbors([input_df_mean])

/Users/felipedalcin/.pyenv/versions/3.10.6/envs/WineGen/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [324]:
distances

array([[  0.42628996,   0.56476438,   0.56602123, ...,  98.52543756,
         98.54575987, 130.52419825]])

In [325]:
pd.DataFrame(distances[0]).iloc[input_list]

,0
482,1.014663
33,0.743902
1338,1.216878
106274,5.601531
8090,1.756731


In [326]:
df.iloc[input_list]

,country,description,points,price,province,title,winery,type,region,variety_adj
ID,,,,,,,,,,
519,US,"Deliciously fruity but also well-structured, t...",91,36.0,California,Prospect 772 2014 The Brat Grenache (Calaveras...,Prospect 772,red,Calaveras County,Grenache
38,Italy,"Inky in color, this wine has plump aromas of r...",86,11.0,Southern Italy,Feudi di San Marzano 2011 I Tratturi Primitivo...,Feudi di San Marzano,red,Puglia,Primitivo
1417,US,"The cool vintage was a challenge, which speaks...",93,38.0,California,Dry Creek Vineyard 2011 Somers Ranch Zinfandel...,Dry Creek Vineyard,red,Dry Creek Valley,Primitivo
123338,Argentina,"Dark, compact and with good lift. The nose is ...",88,16.0,Mendoza Province,Cicchitti 2007 Merlot (Mendoza),Cicchitti,red,Mendoza,Merlot
8737,US,"Tasting sweet and hot, this sure is ripe. It h...",87,85.0,California,Chalk Hill 2009 Merlot-Malbec (Chalk Hill),Chalk Hill,red,Chalk Hill,Merlot-Malbec


In [ ]:
#For Attempt 2 - get 10000 neighs for each input. Only accept if number of intercepts > xxx

In [569]:
neigh2 = NearestNeighbors(n_neighbors=10000, algorithm='ball_tree', n_jobs=-1)
neigh2.fit(X_scaled)

NearestNeighbors(algorithm='ball_tree', n_jobs=-1, n_neighbors=10000)

In [575]:
subsets={}
distances={}
list_subsets=[]
for i, wine in enumerate(input_list):
    distance, subset=neigh2.kneighbors([X_scaled.iloc[wine]])
    subsets[f'subset{i}']=subset
    distances[f'distance{i}']=distance
    list_subsets.append(subset)

/Users/felipedalcin/.pyenv/versions/3.10.6/envs/WineGen/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/felipedalcin/.pyenv/versions/3.10.6/envs/WineGen/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/felipedalcin/.pyenv/versions/3.10.6/envs/WineGen/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/felipedalcin/.pyenv/versions/3.10.6/envs/WineGen/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/felipedalcin/.pyenv/versions/3.10.6/envs/WineGen/lib/python3.10/site-packages/sklearn/base.py:465

In [576]:
combinations_list = []
array_indeces = []
for r in range (2, len(list_subsets)+1):
    combinations_list.extend(combinations(list_subsets, r))
    array_indeces.extend(combinations(range(0,len(list_subsets)),r))

i=0
j=0
intercepts=[]
intercept_wines=[]
for perm in combinations_list:
    for i, arr in enumerate(perm):
        if i ==0:
            arr_ref=arr
        else:
            arr_ref=np.intersect1d(arr_ref, arr)
    intercepts.append(arr_ref.shape[0])
    intercept_wines.append(arr_ref)
    print(array_indeces[j],arr_ref.shape[0])
    j+=1
    print('\n')
wine_combinations={'wines':array_indeces,'wines_used':[len(combo) for combo in array_indeces],'intercept':intercepts,'intercept_wines':intercept_wines}
wine_combinations=pd.DataFrame(wine_combinations)

(0, 1) 0


(0, 2) 3275


(0, 3) 81


(0, 4) 1698


(1, 2) 617


(1, 3) 4363


(1, 4) 7


(2, 3) 2906


(2, 4) 4314


(3, 4) 848


(0, 1, 2) 0


(0, 1, 3) 0


(0, 1, 4) 0


(0, 2, 3) 76


(0, 2, 4) 1220


(0, 3, 4) 14


(1, 2, 3) 563


(1, 2, 4) 7


(1, 3, 4) 7


(2, 3, 4) 709


(0, 1, 2, 3) 0


(0, 1, 2, 4) 0


(0, 1, 3, 4) 0


(0, 2, 3, 4) 14


(1, 2, 3, 4) 7


(0, 1, 2, 3, 4) 0




In [253]:
# for wines_used in range(len(list_subsets),1,-1):
for item in range(len(wine_combinations['wines_used']-1,-1,-1):
    
    

4204

In [444]:
wine_combinations[wine_combinations['wines_used']==4]

,wines,wines_used,intercept,intercept_wines
20,"(0, 1, 2, 3)",4,0,[]
21,"(0, 1, 2, 4)",4,0,[]
22,"(0, 1, 3, 4)",4,0,[]
23,"(0, 2, 3, 4)",4,14,"[4893, 9480, 10688, 19664, 30601, 31994, 38174..."
24,"(1, 2, 3, 4)",4,7,"[6656, 8979, 10619, 22615, 38422, 45723, 74610]"


In [445]:
wine_combinations[wine_combinations['wines_used']==4]['intercept'].max()

14

In [577]:
found_best_match=False
no_matches=False
best_match_row=None
best_single_wine_match=None
for i in range(len(input_list),1,-1):
    if not found_best_match:
        if wine_combinations[wine_combinations['wines_used']==i]['intercept'].max() > 1000:
            best_match_row=wine_combinations.loc[
            (wine_combinations['intercept']==wine_combinations[wine_combinations['wines_used']==i]['intercept'].max())
            &(wine_combinations['wines_used']==i)]
            found_best_match=True
        elif i==2:
            if wine_combinations['intercept'].max()==0:
                no_matches=True
            else:
                best_match_row=wine_combinations.loc[wine_combinations['intercept'].max()]
if no_matches: # If there are not combine matches, it returns the recommendations for a single with the shortest distances.
    total_distance=pd.DataFrame()
    for i, wine in enumerate(distances):
        total_distance=total_distance._append({'distance':distances[wine].sum()}, ignore_index=True)
    best_single_wine_match=total_distance.loc[total_distance['distance']==total_distance['distance'].min()]

In [578]:
unfiltered_wine_list=pd.DataFrame()
match_distances=[]
if best_match_row is not None:
    unfiltered_wine_list['match_wine']=best_match_row['intercept_wines'].values[0]
    unfiltered_wine_list['total_distance']=0
    for wine in best_match_row['wines'].values[0]:
        match_distances=[]
        for match_wine in unfiltered_wine_list['match_wine']:
            match_distances.append(distances_df.iloc[subsets_df.loc[subsets_df[f'subset{wine}']==match_wine].index[0]].iloc[wine])
        unfiltered_wine_list['total_distance']=unfiltered_wine_list['total_distance'] + match_distances
else:
    unfiltered_wine_list['match_wine']=subsets_df[f'subset{best_single_wine_match.index[0]}']
    unfiltered_wine_list['total_distance']=distances_df[f'distance{best_single_wine_match.index[0]}']
unfiltered_wine_list.sort_values(by=['total_distance'], inplace=True)
unfiltered_wine_list.reset_index(inplace=True,drop=True)

In [574]:
unfiltered_wine_list

,match_wine,total_distance
0,18742,4.572466
1,1338,4.572466
2,14914,4.592706
3,35917,4.624287
4,77019,4.651933
...,...,...
1215,68268,6.961153
1216,8922,6.963735
1217,58407,6.968062
1218,39961,6.968062


In [528]:
for wine in unfiltered_wine_list['match_wine']:
    print(wine)

67
132
145
170
174
239
267
438
619
669
1156
1338
2107
2274
2277
2325
2432
2484
2495
2542
3010
3096
3228
3268
3304
3402
3464
3608
3800
3830
3863
3893
3935
4091
4096
4132
4549
4851
4893
5056
5205
5222
5299
5439
5446
5662
5690
5782
6309
6408
6418
6602
6683
6831
6856
7112
7118
7176
7271
7484
7492
7536
7642
7938
7945
7968
8014
8072
8078
8212
8253
8310
8393
8540
8574
8577
8582
8673
8718
8764
8922
8968
9058
9070
9368
9450
9480
9506
9508
9517
9519
9590
9631
9654
9715
9772
9832
9835
9875
10061
10191
10227
10286
10433
10485
10598
10646
10688
10993
11040
11118
11148
11152
11590
11686
11793
11905
12089
12091
12233
12357
12627
12871
12913
12926
13184
13452
13547
13558
13560
13706
13708
13800
13801
13894
13984
14092
14271
14273
14274
14469
14507
14585
14677
14704
14705
14867
14914
14942
15104
15215
15344
15435
15728
15729
15753
15883
15966
16006
16099
16149
16199
16241
16360
16361
16408
16461
16491
16493
16572
16590
16597
16642
16924
16978
16983
17165
17294
17350
17509
17516
17652
17667
17965
17977


In [517]:
distances_df.iloc[subsets_df.loc[subsets_df[f'subset{0}']==67].index[0]]

distance0    1.646426
distance1    1.668016
distance2    1.824228
distance3    1.743623
distance4    2.451413
Name: 4230, dtype: float64

In [500]:
subsets_df={}
for key in subsets:
    subsets_df[key]=subsets[key].flatten()
subsets_df=pd.DataFrame(subsets_df)
subsets_df

,subset0,subset1,subset2,subset3,subset4
0,482,33,18742,106274,8090
1,104198,95701,1338,65042,49166
2,5974,59218,14914,53596,47387
3,55521,3502,8718,71425,99625
4,37071,7471,59435,15400,58409
...,...,...,...,...,...
9995,45853,88503,99909,68915,39409
9996,19663,108558,5427,2996,58246
9997,86947,65922,1351,95549,93196
9998,29961,64873,111246,75809,12640


In [501]:
distances_df={}
for key in distances:
    distances_df[key]=distances[key].flatten()
distances_df=pd.DataFrame(distances_df)
distances_df

,distance0,distance1,distance2,distance3,distance4
0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.201613,0.210909,0.000000,0.201247,0.421395
2,0.409448,0.293398,0.068843,0.221717,0.705389
3,0.423237,0.298557,0.240000,0.223888,0.705389
4,0.474073,0.302389,0.242564,0.253969,0.705389
...,...,...,...,...,...
9995,2.015703,1.997393,2.127388,2.168050,2.924864
9996,2.015730,1.997504,2.127434,2.168053,2.924864
9997,2.015855,1.997507,2.127441,2.168098,2.924944
9998,2.015987,1.997558,2.127468,2.168230,2.925015


In [ ]:
df.iloc[list(wine_combinations['intercept_wines'][14])].to_csv("../raw_data/Jaimes_list.csv", index=False)